In [27]:
from pathlib import Path
import re
import math

In [14]:
# wjets_ht1/cat_ZZ_EC90_boosted_bb_boostedTau/
base = "/grid_mnt/data__data.polcms/cms/cuisset/cmt/Categorization/bul_2018_ZZ_v12/"
preprocess_base = "/grid_mnt/data__data.polcms/cms/cuisset/cmt/PreprocessRDF/bul_2018_ZZ_v12/"
preprocess_tail = "cat_base_selection/prod_241213c"
prod = "prod_241213c"

In [ ]:
file_regex = re.compile("data_[0-9]{1,}.root")
results = {}
for dataset in Path(base).iterdir():
    preprocess_size = 0
    preprocess_n_files = 0
    try:
        for f in (Path(preprocess_base) / dataset.name / preprocess_tail).iterdir():
            if not re.fullmatch(file_regex, f.name):
                continue
            preprocess_size += f.stat().st_size
            preprocess_n_files += 1
        results[dataset.name] = dict(preprocess_size=preprocess_size/(1e6), preprocess_n_files=preprocess_n_files)
    except FileNotFoundError:
        results[dataset.name] = dict()
    for category in ["cat_ZZ_EC90_boosted_bb_boostedTau","cat_ZZ_elliptical_cut_90_sr", "cat_ZZ_EC90_boosted_bb_HPSTau", "cat_ZZ_EC90_resolved_1b_HPSTau", "cat_ZZ_EC90_resolved_2b_HPSTau"]:
        final_dir = dataset / category / prod
        if not final_dir.is_dir():
            #print(final_dir)
            continue
        total_size = 0
        n_files = 0
        for f in final_dir.iterdir():
            if not re.fullmatch(file_regex, f.name):
                continue
            #print(f)
            total_size += f.stat().st_size
            n_files += 1
        
        results[dataset.name][category] = dict(total_size=total_size/(1e6), n_files=n_files)
results # size in MB

['GluGluToXToZZTo2B2Tau_M1000',
 'GluGluToXToZZTo2B2Tau_M1100',
 'GluGluToXToZZTo2B2Tau_M1200',
 'GluGluToXToZZTo2B2Tau_M1300',
 'GluGluToXToZZTo2B2Tau_M1400',
 'GluGluToXToZZTo2B2Tau_M1500',
 'GluGluToXToZZTo2B2Tau_M1600',
 'GluGluToXToZZTo2B2Tau_M1700',
 'GluGluToXToZZTo2B2Tau_M1800',
 'GluGluToXToZZTo2B2Tau_M1900',
 'GluGluToXToZZTo2B2Tau_M200',
 'GluGluToXToZZTo2B2Tau_M2000',
 'GluGluToXToZZTo2B2Tau_M210',
 'GluGluToXToZZTo2B2Tau_M220',
 'GluGluToXToZZTo2B2Tau_M2200',
 'GluGluToXToZZTo2B2Tau_M230',
 'GluGluToXToZZTo2B2Tau_M240',
 'GluGluToXToZZTo2B2Tau_M2400',
 'GluGluToXToZZTo2B2Tau_M250',
 'GluGluToXToZZTo2B2Tau_M2500',
 'GluGluToXToZZTo2B2Tau_M260',
 'GluGluToXToZZTo2B2Tau_M2600',
 'GluGluToXToZZTo2B2Tau_M270',
 'GluGluToXToZZTo2B2Tau_M280',
 'GluGluToXToZZTo2B2Tau_M2800',
 'GluGluToXToZZTo2B2Tau_M300',
 'GluGluToXToZZTo2B2Tau_M3000',
 'GluGluToXToZZTo2B2Tau_M320',
 'GluGluToXToZZTo2B2Tau_M350',
 'GluGluToXToZZTo2B2Tau_M3500',
 'GluGluToXToZZTo2B2Tau_M360',
 'GluGluToXToZZTo2B2T

In [43]:
results

{'dy_ptz6': {'preprocess_size': 1696.673107,
  'preprocess_n_files': 13,
  'cat_ZZ_EC90_boosted_bb_boostedTau': {'total_size': 22.598863,
   'n_files': 13},
  'cat_ZZ_EC90_boosted_bb_HPSTau': {'total_size': 2.423208, 'n_files': 13},
  'cat_ZZ_EC90_resolved_1b_HPSTau': {'total_size': 15.730865, 'n_files': 13},
  'cat_ZZ_EC90_resolved_2b_HPSTau': {'total_size': 3.789002, 'n_files': 13}},
 'GluGluToXToZZTo2B2Tau_M4000': {'preprocess_size': 816.9867,
  'preprocess_n_files': 4,
  'cat_ZZ_EC90_boosted_bb_boostedTau': {'total_size': 199.481287,
   'n_files': 4},
  'cat_ZZ_EC90_boosted_bb_HPSTau': {'total_size': 2.465651, 'n_files': 4},
  'cat_ZZ_EC90_resolved_1b_HPSTau': {'total_size': 0.863504, 'n_files': 4},
  'cat_ZZ_EC90_resolved_2b_HPSTau': {'total_size': 0.780442, 'n_files': 4}},
 'GluGluToXToZZTo2B2Tau_M2000': {'preprocess_size': 671.590943,
  'preprocess_n_files': 4,
  'cat_ZZ_EC90_boosted_bb_boostedTau': {'total_size': 154.396353,
   'n_files': 4},
  'cat_ZZ_EC90_boosted_bb_HPSTau': 

In [44]:
154155.496883/(5*1e3)

30.831099376600005

In [45]:
final_res = {}
for dataset, info in results.items():
    n_files = None
    factor = None
    if "preprocess_size" not in info:
        print(dataset)
        continue
    if info["preprocess_size"] < 1000:
        n_files = 1
        final_res[dataset] = {"":1}
    else:
        for category, cat_info in info.items():
            if not category.startswith("cat_"): continue

            nice_cat_name = category.removeprefix("cat_ZZ_EC90_")

            n_files = max(
                info["preprocess_size"] / (5*1e3), # nb of files such that each is 5 GB of preprocess
                cat_info["total_size"] / 1000, # nb of files such that cat output is 1GB
            )
            n_files = (max(math.ceil(n_files), 1))

            final_res.setdefault(dataset, dict())[nice_cat_name] = n_files
        
        # baseline
        preprocess_average_file_size = info["preprocess_size"] / info["preprocess_n_files"]
        # in case : preprocess input is very small and we are at least merging at least of a factor 2
        if preprocess_average_file_size < 500 and (info["preprocess_size"] / 1000) < 0.5*info["preprocess_n_files"]:
            final_res[dataset]["base"] = max(math.ceil(info["preprocess_size"] / 1000), 1)
final_res
        

data_mutau_d
data_mutau_c
data_mutau_b
data_mutau_a
data_etau_d
data_etau_c
data_etau_b
data_etau_a
data_tau_d
data_tau_c
data_tau_b
data_tau_a


{'dy_ptz6': {'boosted_bb_boostedTau': 1,
  'boosted_bb_HPSTau': 1,
  'resolved_1b_HPSTau': 1,
  'resolved_2b_HPSTau': 1,
  'base': 2},
 'GluGluToXToZZTo2B2Tau_M4000': {'': 1},
 'GluGluToXToZZTo2B2Tau_M2000': {'': 1},
 'GluGluToXToZZTo2B2Tau_M5000': {'': 1},
 'GluGluToXToZZTo2B2Tau_M3000': {'': 1},
 'GluGluToXToZZTo2B2Tau_M1000': {'': 1},
 'GluGluToXToZZTo2B2Tau_M200': {'': 1},
 'tt_dl': {'boosted_bb_boostedTau': 10,
  'boosted_bb_HPSTau': 10,
  'resolved_1b_HPSTau': 16,
  'resolved_2b_HPSTau': 13,
  'base': 47},
 'tt_sl': {'boosted_bb_boostedTau': 31,
  'boosted_bb_HPSTau': 31,
  'resolved_1b_HPSTau': 62,
  'resolved_2b_HPSTau': 36,
  'base': 155},
 'dy': {'boosted_bb_boostedTau': 1,
  'boosted_bb_HPSTau': 1,
  'resolved_1b_HPSTau': 1,
  'resolved_2b_HPSTau': 1,
  'base': 5},
 'dy_ptz5': {'boosted_bb_boostedTau': 1,
  'boosted_bb_HPSTau': 1,
  'resolved_1b_HPSTau': 1,
  'resolved_2b_HPSTau': 1,
  'base': 2},
 'dy_ptz4': {'boosted_bb_boostedTau': 2,
  'boosted_bb_HPSTau': 2,
  'resolved

In [48]:
1290/31

41.61290322580645

In [46]:
for dataset, d in sorted(final_res.items()):
    print(dataset)
    print("categorization_n_outputs=" + str(d) + ",")
    print()

GluGluToXToZZTo2B2Tau_M1000
categorization_n_outputs={'': 1},

GluGluToXToZZTo2B2Tau_M1100
categorization_n_outputs={'': 1},

GluGluToXToZZTo2B2Tau_M1200
categorization_n_outputs={'': 1},

GluGluToXToZZTo2B2Tau_M1300
categorization_n_outputs={'': 1},

GluGluToXToZZTo2B2Tau_M1400
categorization_n_outputs={'': 1},

GluGluToXToZZTo2B2Tau_M1500
categorization_n_outputs={'': 1},

GluGluToXToZZTo2B2Tau_M1600
categorization_n_outputs={'': 1},

GluGluToXToZZTo2B2Tau_M1700
categorization_n_outputs={'': 1},

GluGluToXToZZTo2B2Tau_M1800
categorization_n_outputs={'': 1},

GluGluToXToZZTo2B2Tau_M1900
categorization_n_outputs={'': 1},

GluGluToXToZZTo2B2Tau_M200
categorization_n_outputs={'': 1},

GluGluToXToZZTo2B2Tau_M2000
categorization_n_outputs={'': 1},

GluGluToXToZZTo2B2Tau_M210
categorization_n_outputs={'': 1},

GluGluToXToZZTo2B2Tau_M220
categorization_n_outputs={'': 1},

GluGluToXToZZTo2B2Tau_M2200
categorization_n_outputs={'': 1},

GluGluToXToZZTo2B2Tau_M230
categorization_n_outputs={'': 1